In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('firstAid.json').read()
intents = json.loads(data_file)

## Предварительная обработка данных

Перебор шаблонов и разметка предложений с помощью функции nltk.word_tokenize(). Добавлени каждого слова в список слов. Создани списка классов для тегов.

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #токинезируем слова
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        # добавление слов в массив
        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            #print(classes)

## Лемматизация каждого слова и удаление повторяющихся слов из списка

In [3]:

#Лемматизация, приведение слов к нижнему регистру, удаление дубликатов
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# сортировка 
classes = sorted(list(set(classes)))

# documents = комбинация между паттернами и намерениями
print (len(documents), "documents", "\n")

# classes = намерения
print (len(classes), "classes","\n\n", classes,"\n")

# words = все слова, выражения
print (len(words), "unique lemmatized words","\n\n", words, "\n")

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

188 documents 

44 classes 

 ['Abdonominal Pain', 'Abrasions', 'Broken Toe', 'Bruises', 'CPR', 'Chemical Burn', 'Choking', 'Cold', 'Cough', 'Cuts', 'Diarrhea', 'Drowning', 'Eye Injury', 'Fainting', 'Fever', 'Fracture', 'Frost bite', 'Gastrointestinal problems', 'Head Injury', 'Headache', 'Heat Exhaustion', 'Heat Stroke', 'Insect Bites', 'Nasal Congestion', 'Normal Bleeding', 'Poison', 'Pulled Muscle', 'Rash', 'Rectal bleeding', 'Skin problems', 'Sore Throat', 'Splinter', 'Sprains', 'Strains', 'Sun Burn', 'Teeth', 'Testicle Pain', 'Vertigo', 'Wound', 'animal bite', 'nose bleed', 'seizure', 'snake bite', 'stings'] 

124 unique lemmatized words 

 ['a', 'abdonominal', 'abrasion', 'allergy', 'am', 'an', 'animal', 'apply', 'better', 'bit', 'bite', 'bitten', 'bleed', 'bleeding', 'blocked', 'bring', 'broken', 'bruise', 'burn', 'by', 'cause', 'chemical', 'choke', 'choked', 'choking', 'cold', 'congestion', 'cough', 'cpr', 'cream', 'cure', 'cut', 'diagnose', 'diarrhea', 'do', 'doe', 'dog', 'dro

## Создание обучающих и тестовых данных

In [4]:


# создаем тренировочный массив
training = []

# пустой массив для вывода
output_empty = [0] * len(classes)

# тренировочный набор, набор слов для каждого предложения
for doc in documents:

    # инициируем наш набор слов
    bag = []

    # список маркированных слов для паттерна
    pattern_words = doc[0]

    # Лемматизируем каждое слово - создаем базовое слово, для представления связанных слов
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # добавление в массив 1, если в текущем шаблоне найдено совпадение
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    

    # вывод 0 для каждого тега и 1 для текущего тега (для каждого шаблона)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# перемешивание признаков и преобразование их в np.arry
random.shuffle(training)
training = np.array(training)

# создание тренировочных и тестовых наборов, X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Тренировочные данные подготовленны")

Тренировочные данные подготовленны


C:\Users\sudo\AppData\Local\Temp/ipykernel_7392/3830689120.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


## Обучение модели

In [5]:
# Параметры модели - 3 слоя. Первый слой 128 нейронов, второй слой 64 нейрона и 3 - выводной слой  
# содержит колличество нейронов равных числу равное количеству 
# намерений для прогнозирования выходного намерения с помощью softmax.
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


# Компиляция модели. Используем стохастический градиентный бустинг Нестерова 
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


# подгонка и сохранение модели
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200


C:\Users\sudo\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


38/38 [==============================] - 0s 1ms/step - loss: 3.8178 - accuracy: 0.0213 
Epoch 2/200
38/38 [==============================] - 0s 965us/step - loss: 3.7715 - accuracy: 0.0479
Epoch 3/200
38/38 [==============================] - 0s 979us/step - loss: 3.7695 - accuracy: 0.0319
Epoch 4/200
38/38 [==============================] - 0s 1ms/step - loss: 3.7171 - accuracy: 0.0691
Epoch 5/200
38/38 [==============================] - 0s 979us/step - loss: 3.6910 - accuracy: 0.0691
Epoch 6/200
38/38 [==============================] - 0s 1ms/step - loss: 3.6003 - accuracy: 0.1011
Epoch 7/200
38/38 [==============================] - 0s 1ms/step - loss: 3.5307 - accuracy: 0.1277
Epoch 8/200
38/38 [==============================] - 0s 979us/step - loss: 3.4573 - accuracy: 0.1277
Epoch 9/200
38/38 [==============================] - 0s 1ms/step - loss: 3.2980 - accuracy: 0.1968
Epoch 10/200
38/38 [==============================] - 0s 1ms/step - loss: 3.1446 - accuracy: 0.2553
Epoch 11/200

38/38 [==============================] - 0s 1ms/step - loss: 0.4054 - accuracy: 0.8883
Epoch 83/200
38/38 [==============================] - 0s 925us/step - loss: 0.3670 - accuracy: 0.8777
Epoch 84/200
38/38 [==============================] - 0s 912us/step - loss: 0.3152 - accuracy: 0.9202
Epoch 85/200
38/38 [==============================] - 0s 858us/step - loss: 0.2661 - accuracy: 0.9468
Epoch 86/200
38/38 [==============================] - 0s 885us/step - loss: 0.4135 - accuracy: 0.8617
Epoch 87/200
38/38 [==============================] - 0s 898us/step - loss: 0.3598 - accuracy: 0.8723
Epoch 88/200
38/38 [==============================] - 0s 898us/step - loss: 0.3322 - accuracy: 0.9043
Epoch 89/200
38/38 [==============================] - 0s 885us/step - loss: 0.2759 - accuracy: 0.9202
Epoch 90/200
38/38 [==============================] - 0s 938us/step - loss: 0.3496 - accuracy: 0.9096
Epoch 91/200
38/38 [==============================] - 0s 912us/step - loss: 0.3698 - accuracy: 0.

38/38 [==============================] - 0s 912us/step - loss: 0.1780 - accuracy: 0.9468
Epoch 162/200
38/38 [==============================] - 0s 912us/step - loss: 0.2565 - accuracy: 0.9202
Epoch 163/200
38/38 [==============================] - 0s 871us/step - loss: 0.2251 - accuracy: 0.9309
Epoch 164/200
38/38 [==============================] - 0s 885us/step - loss: 0.3223 - accuracy: 0.9202
Epoch 165/200
38/38 [==============================] - 0s 898us/step - loss: 0.1537 - accuracy: 0.9468
Epoch 166/200
38/38 [==============================] - 0s 898us/step - loss: 0.1997 - accuracy: 0.9309
Epoch 167/200
38/38 [==============================] - 0s 885us/step - loss: 0.2324 - accuracy: 0.9415
Epoch 168/200
38/38 [==============================] - 0s 938us/step - loss: 0.1222 - accuracy: 0.9574
Epoch 169/200
38/38 [==============================] - 0s 925us/step - loss: 0.1842 - accuracy: 0.9521
Epoch 170/200
38/38 [==============================] - 0s 912us/step - loss: 0.1934 - a

In [6]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('firstAid.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [7]:
def clean_up_sentence(sentence):

    # Маркеруем шаблоны - разделяем слова на массив
    sentence_words = nltk.word_tokenize(sentence)

    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# возвращаем набор слов: 0 или 1 для каждого слова, существующих в предложении 

In [8]:
def bow(sentence, words, show_details=True):

    # Маркеруем шаблон
    sentence_words = clean_up_sentence(sentence)

    # Набор слов - матрица из N слов
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 

                # присваиваем 1 если слово находится в словарной позиции
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [9]:
def predict_class(sentence, model):

    # отфильтровать прогнозы ниже порогового значения
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

    # сортировка по степени вероятности
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [10]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [11]:
def chatbot_response(text):
    # Проверка особых случаев
    try:
        ints = predict_class(text, model)
        res = getResponse(ints, intents)
        
        if res == " ":
            return "Sorry. I don't know"
    except:
        return "Sorry. I don't understand :("
    return res

## Интерфейс чата

In [12]:
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
        
base = Tk()
base.title("First Aid Chat")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)



# Создание диалогового окна
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)


# Привязка полосы прокрутки к окну чата
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set


# Создание кнопки отправки сообщений
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )


# Создание поля ввода сообщения
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial", background="#dddddd")



# Координаты расположения компонентов на экране
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=130, y=401, height=40, width=265)
SendButton.place(x=6, y=401, height=40)

In [13]:
base.mainloop()

1/1 [==============================] - 0s 15ms/step
